In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, MinuteLocator, SecondLocator
import numpy as np
import datetime
import os
from omegaconf import OmegaConf, DictConfig, ListConfig

import geopandas as gpd
from shapely.geometry import Point

from sklearn.cluster import DBSCAN
from sklearn.cluster import OPTICS

from sklearn.metrics.pairwise import haversine_distances

from geopy.distance import great_circle

from shapely.geometry import MultiPoint

from tqdm import tqdm


from hydra import initialize, compose
from omegaconf import OmegaConf

with initialize(version_base=None, config_path="..\\scripts\\config\\"):
    cfg = compose(config_name='config.yaml')

In [3]:
def get_slot(x, slot_len):
    hm = pd.to_datetime(x, unit='s').dt.strftime('%H:%M').str.split(':')
    return hm.apply(lambda x: (60 * int(x[0]) + int(x[1])) // slot_len)


launch_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
rng = np.random.default_rng(seed=cfg.seed)

if isinstance(cfg.id_points_range, ListConfig):
    bounds = OmegaConf.to_container(cfg.id_points_range)
else:
    if cfg.id_points_range >= 0:
        bounds = [0, cfg.id_points_range]
    else:
        bounds = [0, np.inf]

locs = pd.read_csv(rf"C:\Users\kirin\jupyter\CityTraffic\{cfg.locs_path}")

if cfg.n_ids == -1:
    ID_LIST = pd.unique(locs['id'])
elif isinstance(cfg.n_ids, ListConfig):
    ID_LIST = OmegaConf.to_container(cfg.n_ids)
else:
    ID_LIST = rng.choice(pd.unique(locs['id']), size=cfg.n_ids)


In [4]:
new_id_df = pd.DataFrame(columns=locs.columns)
for cur_id in tqdm(ID_LIST):
        cur_id_df = locs.query('id == @cur_id').copy()
        if cur_id_df['cnt'].sum() > bounds[1] or cur_id_df['cnt'].sum() < bounds[0]:
            continue
        cur_id_df['length'] = cur_id_df['last_ts'] - cur_id_df['first_ts']
        #     print(cur_id_df['length'].sum())
        days = cur_id_df.groupby('log_date')['length'].sum()
        days = days[days >= cfg.min_time_day_thresh]
        if days.shape[0] >= cfg.min_days_thresh:
            filtered_data = cur_id_df[cur_id_df['log_date'].isin(days.index)]
            new_id_df = pd.concat([new_id_df, filtered_data])

  1%|          | 1280/151891 [02:14<3:33:25, 11.76it/s]

In [ ]:
new_id_df.shape